# Used cars on German eBay

For this project, we will be working with some data that was scraped and uploaded to Kaggle. There are 50,000 data points we will be cleaning and analyzing. There are quite a few Series in this data set, so naturally there will be much to clean!

To get started, let's import our libraries:

In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#autos = pd.read_csv('autos.csv')

You will notice that I've commented out the `pd.read_csv()`. When attempting to load the file, I encountered an encoding error. It seems that UTF-8, the most common encoding method for these filetypes (and what pandas uses as a default argument for the encoding parameter), was unsuccessful. Let's try a different encoding method and see if it is a success.

In [2]:
autos = pd.read_csv('autos.csv',encoding='Latin-1')

In [3]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


*Success!* This particular csv file was using a `Latin-1` encoding method. Using this argument, we were able to load in our Dataframe.

In [4]:
autos.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

## What's the info?
Taking a look at the list, we can immediately see a few issues. There are columns with null data. Specifically, the *vehicleType*, *gearbox*, *model*, *fuelType*, and *notRepairedDamage* columns. Below, I created a variable to isolate the null values in the *gearbox column*.

In [5]:
a_gear_missing = autos[autos['gearbox'].isnull()]

In [6]:
#df of NaN values in the gearbox column
a_gear_missing['vehicleType'].value_counts()

kleinwagen    403
limousine     306
kombi         233
bus           138
cabrio        110
coupe          62
suv            53
andere         32
Name: vehicleType, dtype: int64

## The headers

The column headers are currently in camelCase, which we don't want to work with. Instead, I'd like to convert these all to snake_case! Manually editing each line is one way of doing it, but that'd take too much time. And besides, what's programming good for if we can't do it the lazy way? ;)

Let's start by creating a function that uses a regex to convert the camelCase to snake_case and return a nicely formatted list that we will then assign to our `autos.columns`!

In [7]:
columns = autos.columns

In [8]:
def snake_me(col):
    a = re.compile('((?<=[a-z0-9])[A-Z]|(?!^)[A-Z](?=[a-z]))')
    new_list = []
    for i in col:
        i = a.sub(r'_\1',i).lower()
        new_list.append(i)
    return new_list
        
        
    

new_col = snake_me(columns)
new_col
        

['date_crawled',
 'name',
 'seller',
 'offer_type',
 'price',
 'abtest',
 'vehicle_type',
 'year_of_registration',
 'gearbox',
 'power_ps',
 'model',
 'odometer',
 'month_of_registration',
 'fuel_type',
 'brand',
 'not_repaired_damage',
 'date_created',
 'nr_of_pictures',
 'postal_code',
 'last_seen']

Very nice! There are a few edits that I'd like to make to some of the column names, however. Now, we *could* make individual edits using `str.replace()` in a bunch of separate calls, but again... ***WHY?***. Let's use a nice fancy list comprehension instead!

In [9]:
new_col = [n.replace('year_of_registration','registration_year').replace('month_of_registration','registration_month').replace('not_repaired_damage','unrepaired_damage').replace('date_created','ad_created') for n in new_col]
    

In [10]:
autos.columns = new_col

In [11]:
autos.head(2)

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08


## And just like that!

We've replaced the string values we wanted and have them all nicely formatted. Looking back, I should have replaced the values first and then ran it through the function to format the camelCase to snake_case, but what's learning if you don't make any mistakes? Taking a look at the column headers, we can see that our goal has been met!

## Let's explore our data next, shall we?

We've done some preliminary work, but the fun has just begun! What we want to do next is clean up some columns. Let's use the `.describe()` method to gain some insights. We will use the `include='all'` argument to gain both categorical and numeric columns.

In [12]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-27 22:55:05,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Taking a look, there are a few things that pop out. One, the *odometer* column can be adjusted by removing the 'km' and converting the entire Series to numeric data in order to perform aggregations on them. The *price* colum is also not numeric data as there are dollar signs galore. Let's fix this, shall we?

In [13]:
odom = autos['odometer']

In [14]:
odom = [o.replace('km','').replace('k','').replace(',','') for o in odom]

In [15]:
autos['odometer'] = odom
autos['odometer'] = autos['odometer'].astype(int)

In [16]:
autos.rename(columns={'odometer':'odometer_km'},inplace=True)
autos['odometer_km']

0        150000
1        150000
2         70000
3         70000
4        150000
          ...  
49995    100000
49996    150000
49997      5000
49998     40000
49999    150000
Name: odometer_km, Length: 50000, dtype: int64

## Odometer is now an int64 dtype!

And while we were at it, we removed any unwanted characters like *km* or *k*, and renamed the column from *odometer* to *odometer_km* for clarity. To avoid doing the whole `autos['odometer'] = autos.rename(columns={'odometer':'odometer_km'})`, we just used the `inplace=True` argument to push our changes to the DataFrame more succinctly.

### Now, getting back to the rest of the data set...

Let's explore what some columns entail. As we clean this data, we need to ask ourselves if there are any outliers, inconsistencies, or impossibilities within certain Series. We will explore our newly formatted ***odometer_km*** Series. Let's use a couple of attributes and methods to explore!

In [17]:
odom = autos['odometer_km']
odom_shape = odom.unique().shape
odom_desc = odom.describe()
odom_value = odom.value_counts().sort_index()

In [18]:
odom_shape[0]

13

In [19]:
odom_desc

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [20]:
odom_value

5000        967
10000       264
20000       784
30000       789
40000       819
50000      1027
60000      1164
70000      1230
80000      1436
90000      1757
100000     2169
125000     5170
150000    32424
Name: odometer_km, dtype: int64

In [21]:
price = autos['price']

remove_dol = re.compile(r'[$,]')

stripped_price = []

for p in price:
    stripped = re.sub(remove_dol,"",p)
    p = float(stripped)
    stripped_price.append(p)
    
autos['price'] = stripped_price
autos.columns = (a.replace('price','price_dol') for a in autos.columns)


In [22]:
price = autos['price_dol'].value_counts()


In [23]:
autos.dtypes

date_crawled           object
name                   object
seller                 object
offer_type             object
price_dol             float64
abtest                 object
vehicle_type           object
registration_year       int64
gearbox                object
power_ps                int64
model                  object
odometer_km             int64
registration_month      int64
fuel_type              object
brand                  object
unrepaired_damage      object
ad_created             object
nr_of_pictures          int64
postal_code             int64
last_seen              object
dtype: object

In [24]:
autos['seller'].value_counts(normalize=True)

privat        0.99998
gewerblich    0.00002
Name: seller, dtype: float64

In [25]:
time_info = autos[['date_crawled','ad_created','last_seen']].head()

In [26]:
d_crawl_per = autos['date_crawled'].str[:10].value_counts(normalize=True,dropna=False).sort_index()
a_create_per = autos['ad_created'].str[:10].value_counts(normalize=True,dropna=False).sort_index()
l_seen_per = autos['last_seen'].str[:10].value_counts(normalize=True,dropna=False).sort_index()

print(f'{d_crawl_per}\n\n{a_create_per}\n\n{l_seen_per}')

2016-03-05    0.02538
2016-03-06    0.01394
2016-03-07    0.03596
2016-03-08    0.03330
2016-03-09    0.03322
2016-03-10    0.03212
2016-03-11    0.03248
2016-03-12    0.03678
2016-03-13    0.01556
2016-03-14    0.03662
2016-03-15    0.03398
2016-03-16    0.02950
2016-03-17    0.03152
2016-03-18    0.01306
2016-03-19    0.03490
2016-03-20    0.03782
2016-03-21    0.03752
2016-03-22    0.03294
2016-03-23    0.03238
2016-03-24    0.02910
2016-03-25    0.03174
2016-03-26    0.03248
2016-03-27    0.03104
2016-03-28    0.03484
2016-03-29    0.03418
2016-03-30    0.03362
2016-03-31    0.03192
2016-04-01    0.03380
2016-04-02    0.03540
2016-04-03    0.03868
2016-04-04    0.03652
2016-04-05    0.01310
2016-04-06    0.00318
2016-04-07    0.00142
Name: date_crawled, dtype: float64

2015-06-11    0.00002
2015-08-10    0.00002
2015-09-09    0.00002
2015-11-10    0.00002
2015-12-05    0.00002
               ...   
2016-04-03    0.03892
2016-04-04    0.03688
2016-04-05    0.01184
2016-04-06    0.00

If we take a look below, we can see a few strange things in our *registration_year* series. First of all, the min is the year 1000. Cars were not invented then, so how could they possibly have been registered then? On the flipside, we've got the max telling us that a car was registered in the year 9999. Unless there is some strange rip in the fabric of time, this is impossible as well. We should clean this up, and determine if some rows need to be deleted or adjusted. Lets take a deep dive into the data.

In [27]:
autos['registration_year'].describe()

count    50000.000000
mean      2005.073280
std        105.712813
min       1000.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

In [28]:
reg_impossible = autos.loc[(autos['registration_year'] <= 1886) | (autos['registration_year']>= 2021)]
reg_impossible[['name','registration_year','vehicle_type']].sort_values('registration_year')

,name,registration_year,vehicle_type
22316,VW_Kaefer.__Zwei_zum_Preis_von_einem.,1000,NaN
49283,Citroen_HY,1001,NaN
24511,Trabant__wartburg__Ostalgie,1111,NaN
35238,Suche_Skoda_Fabia____Skoda_Fabia_Combi_mit_Klima,1500,NaN
32585,UNFAL_Auto,1800,NaN
10556,UNFAL_Auto,1800,NaN
27578,VW_GOLF_2_SYNCRO,2800,NaN
4549,Kompressor,4100,NaN
453,Armee_Jeep,4500,NaN
42079,APE_50___deutsche_Papiere_!!!,4800,NaN


### Houston, we've found the problem 🚀

This time-space phenomenon seems to affect any and all types of cars. This also seems to be a very small amount of data in the grand scheme of things, so let's drop it from the data set.

In [29]:
autos.head()

,date_crawled,name,seller,offer_type,price_dol,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,5000.0,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,8500.0,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,8990.0,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,4350.0,control,kleinwagen,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,1350.0,test,kombi,2003,manuell,0,focus,150000,7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [30]:
# Here we are using the variable which contains the boolean index we used to find all of these outliers.
# Must specify rows with the .index attribute, set 'inplace' to true in order to push changes to original dataframe
autos.drop(reg_impossible.index,inplace=True)

In [31]:
autos.sort_values('registration_year')

,date_crawled,name,seller,offer_type,price_dol,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
3679,2016-04-04 00:36:17,Suche_Auto,privat,Angebot,1.0,test,NaN,1910,NaN,0,NaN,5000,0,NaN,sonstige_autos,NaN,2016-04-04 00:00:00,0,40239,2016-04-04 07:49:15
30781,2016-03-25 13:47:46,Opel_Calibra_V6_DTM_Bausatz_1:24,privat,Angebot,30.0,test,NaN,1910,NaN,0,calibra,100000,0,NaN,opel,NaN,2016-03-25 00:00:00,0,47638,2016-03-26 23:46:29
45157,2016-03-11 22:37:01,Motorhaube,privat,Angebot,15.0,control,NaN,1910,NaN,0,NaN,5000,0,NaN,trabant,NaN,2016-03-11 00:00:00,0,90491,2016-03-25 11:18:57
22659,2016-03-14 08:51:18,Opel_Corsa_B,privat,Angebot,500.0,test,NaN,1910,NaN,0,corsa,150000,0,NaN,opel,NaN,2016-03-14 00:00:00,0,52393,2016-04-03 07:53:55
15898,2016-03-08 10:50:05,Tausch_alles_aus_meinen_Anzeigen_gegen_Auto,privat,Angebot,0.0,test,NaN,1910,NaN,0,NaN,5000,0,NaN,sonstige_autos,NaN,2016-03-08 00:00:00,0,6108,2016-03-08 17:47:19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17928,2016-03-17 19:39:20,Ford_Focus_Ghia_TÜV_3/18_Zahnriemen_Kupplung_W...,privat,Angebot,1849.0,control,NaN,2018,manuell,101,focus,100000,5,benzin,ford,nein,2016-03-17 00:00:00,0,1454,2016-03-24 15:18:23
10891,2016-03-11 09:54:09,2_5_TDI_PASSAT_6V,privat,Angebot,2000.0,control,NaN,2018,manuell,150,passat,150000,9,NaN,volkswagen,nein,2016-03-11 00:00:00,0,64711,2016-03-25 17:17:26
38342,2016-03-07 18:52:22,Suche_Auto_fuer_die_jagd_zum_tauschen,privat,Angebot,0.0,control,NaN,2019,NaN,0,NaN,150000,0,NaN,sonstige_autos,NaN,2016-03-07 00:00:00,0,18510,2016-04-06 13:17:20
49185,2016-03-28 11:45:31,Alfa_Romeo_156_Sportwagon_1_8_T_Spark,privat,Angebot,1200.0,control,NaN,2019,manuell,140,156,150000,5,benzin,alfa_romeo,nein,2016-03-28 00:00:00,0,91058,2016-04-06 13:45:56


# Aggregate Price by Brand

So what's next for our project? Let's find the mean price for the top 6 car manufacturers! We can start by creating an empty dictionary in order to hold our brands and their mean price. I'm taking the top 6 most popular brands in this dataset, and using their index as the key, and mean price as the value.

In [86]:
a_brand_u = autos['brand'].value_counts(normalize=True,dropna=False).sort_values(ascending=False)
print(a_brand_u[:6])
brand_tags = a_brand_u.index

volkswagen       0.213743
opel             0.109212
bmw              0.108612
mercedes_benz    0.094685
audi             0.085701
ford             0.069613
Name: brand, dtype: float64


In [39]:
price_brand = {}

for i in brand_tags[:6]:
    av_price = autos.loc[autos['brand'] == i, 'price_dol'].mean()
    
    price_brand[i] = round(av_price,2)

price_brand = dict(sorted(price_brand.items(), key=lambda item: item[1],reverse=True))

price_brand



{'mercedes_benz': 29511.58,
 'audi': 8965.56,
 'bmw': 8254.44,
 'ford': 7105.66,
 'volkswagen': 6385.35,
 'opel': 5104.25}

### Our new dictionary tells us that the *Mercedez Benz* brand of cars in the top 6 is the most expensive, and the *Renault* brand the least so!

### So, what's next?

Next, we want to aggregate even more data. Below is code that uses the same for loop technique to find the average mileage for our top 6 brands. We're going to take it one step further however, and convert each of our new dictionaries into pandas Series objects, then append those Series objects into a new dataframe in order to view our findings.

In [40]:
brand_mean_ser = pd.Series(price_brand)

In [41]:
brand_mean_ser

mercedes_benz    29511.58
audi              8965.56
bmw               8254.44
ford              7105.66
volkswagen        6385.35
opel              5104.25
dtype: float64

In [47]:
new_df = pd.DataFrame(brand_mean_ser,columns=['brand_mean_price'])

In [44]:
new_df

,brand_mean_price
mercedes_benz,29511.58
audi,8965.56
bmw,8254.44
ford,7105.66
volkswagen,6385.35
opel,5104.25


In [55]:
mean_mileage = {}

for i in brand_tags[:6]:
    av_mile = autos.loc[autos['brand']==i, 'odometer_km'].mean()
    mean_mileage[i] = av_mile

mean_mileage = dict(sorted(mean_mileage.items(), key=lambda item:item[1],reverse=True))

mean_mileage

{'bmw': 132544.21518054532,
 'mercedes_benz': 130937.235841082,
 'audi': 129643.9411627364,
 'opel': 129365.15207035544,
 'volkswagen': 129012.35723647257,
 'ford': 124131.93446392642}

In [80]:
s_mean_mileage = pd.Series(mean_mileage)

new_df['mean_mileage'] = round(s_mean_mileage,2)

In [83]:
new_df['price_per_mile'] = round(new_df['brand_mean_price'] / new_df['mean_mileage'],2)

In [84]:
new_df

,brand_mean_price,mean-mileage,mean_mileage,price_per_mile
mercedes_benz,29511.58,130937.24,130937.24,0.23
audi,8965.56,129643.94,129643.94,0.07
bmw,8254.44,132544.22,132544.22,0.06
ford,7105.66,124131.93,124131.93,0.06
volkswagen,6385.35,129012.36,129012.36,0.05
opel,5104.25,129365.15,129365.15,0.04


# Findings

While there aren't too many differences in our price per mile, we can see that *Mercedez Benz* is by far the most expensive. Aside from it, all the other cars seem to have very similar patterns, almost negligible even. But we were able to determine that from using aggregation techniques on this dataset.

- We cleaned the data and discovered new trends from it!